In [1]:
import os
import time
import pandas as pd
from powerbiclient import Report, models
from powerbiclient.authentication import InteractiveLoginAuthentication
device_auth = InteractiveLoginAuthentication()

A local browser window will open for interactive sign in.

Interactive authentication successfully completed.
You are now logged in.

The result should be passed only to trusted code in your notebook.


In [4]:
import re

url = "https://app.powerbi.com/groups/653fc201-df39-4f8d-9bb1-b9de4e2b03be/reports/6f1bd60a-d7a2-41b8-9850-00c0b0d2c01e/ReportSection76c409e0c333d60bb1e2?experience=power-bi"

pattern = re.compile(r'/groups/(\w+-\w+-\w+-\w+-\w+)/reports/(\w+-\w+-\w+-\w+-\w+)/')
matches = pattern.search(url)

if matches:
    group_id = matches.group(1)
    report_id = matches.group(2)
    print(f"Group ID - {group_id}")
    print(f"Report ID - {report_id}")


Group ID - 653fc201-df39-4f8d-9bb1-b9de4e2b03be
Report ID - 6f1bd60a-d7a2-41b8-9850-00c0b0d2c01e


In [2]:
#Revenue_Opportunities
group_id = "653fc201-df39-4f8d-9bb1-b9de4e2b03be"
report_id = "8bbfe108-f5ca-4579-91a0-9a2d37bdefb4"

In [3]:
report = Report(group_id = group_id, report_id = report_id, auth = device_auth)
report

Report()

In [ ]:
Main - https://app.powerbi.com/groups/653fc201-df39-4f8d-9bb1-b9de4e2b03be/list?experience=power-bi
Report - https://app.powerbi.com/groups/653fc201-df39-4f8d-9bb1-b9de4e2b03be/reports/6f1bd60a-d7a2-41b8-9850-00c0b0d2c01e/ReportSection76c409e0c333d60bb1e2?experience=power-bi
Export - https://app.powerbi.com/reportEmbed?reportId=6f1bd60a-d7a2-41b8-9850-00c0b0d2c01e&autoAuth=true&ctid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61
export2 https://app.powerbi.com/reportEmbed?reportId=6f1bd60a-d7a2-41b8-9850-00c0b0d2c01e&autoAuth=true&ctid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61

In [ ]:
https://app.powerbi.com/reportEmbed?reportId=8bbfe108-f5ca-4579-91a0-9a2d37bdefb4&autoAuth=true&ctid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61
https://app.powerbi.com/reportEmbed?reportId=8bbfe108-f5ca-4579-91a0-9a2d37bdefb4&autoAuth=true&ctid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61

In [53]:
report_pages=report.get_pages()
Page_id = []
Page_Name = []
Visual_ID = []
Visualization = []
Title = []

for report_page in report_pages:
    if report_page['visibility'] == 0 and report_page['isActive'] == True:
        #print(f"Page_id: {report_page['name']}, Page_Name: {report_page['displayName']}, Visibility: {report_page['visibility']}")
        Page_id.append(report_page['name'])
        Page_Name.append(report_page['displayName'])
        report_visuals=report.visuals_on_page(report_page["name"])
        for report_visual in report_visuals:
            try:
                title = report_visual['title']
            except:
                title = None

            #print(f"Visual ID: {report_visual['name']}, Visualization Type:{report_visual['type']}, Title: {title}")
            Visual_ID.append(report_visual['name'])
            Visualization.append(report_visual['type'])
            Title.append(title)
            

Page_id = Page_id * len(Visual_ID)
Page_Name = Page_Name * len(Visual_ID)
data = list(zip(Page_id, Page_Name, Visual_ID, Visualization, Title))
df = pd.DataFrame(data, columns=["Page_id", "Page_Name", "Visual_ID", "Visualization", "Title"])
#df.to_csv(f"POC_Revenue Opportunities_{report_page['displayName']}_VisualMetaData.csv")

In [54]:
dont_consider = ["actionButton", "basicShape", "textbox"]
df2 = df[~df["Visualization"].isin(dont_consider)]
df2

,Page_id,Page_Name,Visual_ID,Visualization,Title
0,ReportSection9380eec50a2a0165bb4b,Detail,df4f5199190e7a2291de,pivotTable,Matrix


In [55]:
df3 = pd.concat([df1, df2], ignore_index = True)
#df3.to_csv("POC_Revenue Opportunities_VisualMetaData.csv")

In [56]:
df3

,Page_id,Page_Name,Visual_ID,Visualization,Title
0,ReportSection,Opportunity Overview,0fa4715079a4d5b14dc6,card,Revenue
1,ReportSection,Opportunity Overview,2a48b7b9c411369725ca,card,Count
2,ReportSection,Opportunity Overview,77960fe4d6dd7e864e6c,slicer,Product Slicer
3,ReportSection,Opportunity Overview,eaf58d9fd08140688010,treemap,Opportunity Count by Region
4,ReportSection,Opportunity Overview,91b894da4c7a504a0d7c,filledMap,Revenue by State
5,ReportSection,Opportunity Overview,efc42a2f1a28997b4092,lineStackedColumnComboChart,Revenue and Opportunity Count by Month
6,ReportSection,Opportunity Overview,772b1906500006b78011,barChart,Pipeline by Stage
7,ReportSection,Opportunity Overview,8a2c970418c8c8b490ba,barChart,Revenue by Region and State
8,ReportSection9380eec50a2a0165bb4b,Detail,df4f5199190e7a2291de,pivotTable,Matrix


In [67]:
Page_id_list = list(df3["Page_id"])
Visual_ID_list = list(df3["Visual_ID"])
Page_Name_list = list(df3["Page_Name"])
Title_list = list(df3["Title"])
Visualization_list = list(df3["Visualization"])

In [163]:
folder_name = "POC_Revenue_Opportunities"
main_column_list = []
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
for i in range(len(Page_id_list)):
    try:
        viz_data=report.export_visual_data(Page_id_list[i],Visual_ID_list[i])
        file_path = os.path.join(folder_name, f"POC_Revenue_Opportunities_{Page_Name_list[i]}_{Title_list[i]}.csv")
        with open(file_path, "w") as file:
            file.write(viz_data)
        dummy2 = pd.read_csv(file_path)
        if "Unnamed: 0" in dummy2:
            dummy2.drop(columns = "Unnamed: 0", axis = 1, inplace = True)
        dummy3_columns = dummy2.columns.tolist()
        main_column_list.append(dummy3_columns)
        dummy2.to_csv(file_path)
    except:
        pass

In [253]:
main_column_list

[['Opportunity Size',
  'Region',
  'Partner',
  'Opportunities',
  'Revenue',
  'Factored Revenue',
  'Avg Revenue']]

In [114]:
main_column_list2 = [', '.join(map(str, sublist)) for sublist in main_column_list]
main_column_series = pd.Series(main_column_list2)
columns_for_description = list(zip(Title_list, Visualization_list, main_column_series))
df4 = pd.DataFrame(columns_for_description, columns = ["Title", "Type", "Columns"])

In [162]:
df4.to_csv("POC_Revenue_Opportunities_MasterData.csv")

In [235]:
df4

,Title,Type,Columns
0,Revenue,card,Opportunity Revenue
1,Count,card,Opportunity Count
2,Product Slicer,slicer,Product Code
3,Opportunity Count by Region,treemap,"Region, State, Opportunity Count"
4,Revenue by State,filledMap,"State, Revenue"
5,Revenue and Opportunity Count by Month,lineStackedColumnComboChart,"Month, Revenue, Opportunity Count"
6,Pipeline by Stage,barChart,"Sales Stage, Opportunity Count"
7,Revenue by Region and State,barChart,"Region, State, Revenue"


## LLM

In [127]:
#Existing credentials
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://azureopenaitext.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key ="6ad08c8e58dd4de9985b86b1209d9cc2"

In [189]:
#Nitin credentials
import os
import openai
openai.api_type = "azure"
openai.api_base ="https://llmdemo.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key ="7ae7bffcd56d4ee89f742dcf20b67269"